# Opencaching + openrouteservice example

Install dependencies:

```sh
pip install folium shapely openrouteservice ortools
```

We set the start point to be somewhere in Berlin.

In [1]:
from context import opencaching

start_coord = (52.518848, 13.399411)

Next we will use the opencaching api, and find the 50 nearest geocaches.

In [2]:
start_lat, start_lon = start_coord
geocaches = opencaching.parse_results(opencaching.search(start_lat, start_lon, count=50))

print(f"Found {len(geocaches)} geocaches near {start_coord}")

Found 50 geocaches near (52.518848, 13.399411)


We can use folium to visualize the geocaches on a map.

In [16]:
import folium
m = folium.Map(tiles='Stamen Toner',location=start_coord, zoom_start=14)

for cache in geocaches:
    folium.map.Marker(
        location=[cache['latitude'], cache['longitude']],
        popup=f"<strong>{cache['code']}</strong><br>Lat:&nbsp;{cache['latitude']:.3f}<br>Lon:&nbsp;{cache['longitude']:.3f}",
        icon=folium.map.Icon(color='darkred'), tooltip=cache['code']
    ).add_to(m)

m

Next, we will use the openrouteservice distance_matrix api to get the distances between all 50 geocaches.

In [17]:
from openrouteservice import client, distance_matrix

clnt = client.Client(key='5b3ce3597851110001cf6248b72014cc8ef14af6b3f38d711224981d')

locations = [(cache['longitude'], cache['latitude']) for cache in geocaches]

locs_str = f"[[{locations[0][0]},{locations[0][1]}]"
for location in locations:
    locs_str += f",[{location[0]},{location[1]}]"
locs_str += "]"
# locs_str

In [18]:
request = {'locations': locations,
           'profile': 'foot-walking',
           'metrics': ['distance']}
    
geocache_matrix = clnt.distance_matrix(**request)

print("Calculated {}x{} routes.".format(len(geocache_matrix['distances']),len(geocache_matrix['distances'][0])))

Calculated 50x50 routes.


Next, we use [ortools](https://developers.google.com/optimization) to find an optimal route between the geocaches. This is solving as a TSP problem.

In [19]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2


def create_distance_callback(data, manager):
    distances_ = data
    index_manager_ = manager
    
    def distance_callback(from_index, to_index):
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = index_manager_.IndexToNode(from_index)
        to_node = index_manager_.IndexToNode(to_index)
        return distances_[from_node][to_node]

    return distance_callback


tsp_size = len(locations)
num_routes = 1
start = 0 # arbitrary start location

if tsp_size > 0:
    manager = pywrapcp.RoutingIndexManager(tsp_size, num_routes, start)
    routing = pywrapcp.RoutingModel(manager)

    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    distance_callback = create_distance_callback(geocache_matrix['distances'], manager)
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    # Solve, returns a solution if any.
    assignment = routing.SolveWithParameters(search_parameters)

optimal_coords = []

def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    print(f"Objective: {assignment.ObjectiveValue()}")
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += node_to_str(manager.IndexToNode(index)) + ' -> '
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        optimal_coords.append(locations[manager.IndexToNode(index)])
    plan_output += node_to_str(manager.IndexToNode(index)) + '\n'
    plan_output += 'Distance of the route: {}m\n'.format(route_distance)
    print(plan_output)

def node_to_str(index):
    return f"{geocaches[index]['code']} {index}"

if assignment:
    print_solution(manager, routing, assignment)

Objective: 28990
Route:
OC146FF 0 -> OC14ADF 3 -> OC13607 8 -> OC13E4C 10 -> OC423D 9 -> OC1206F 1 -> OC13B94 2 -> OC1476C 6 -> OCBD94 13 -> OC13098 29 -> OCFBFA 15 -> OC12E3B 14 -> OC131FA 16 -> OC13B95 42 -> OC15A7F 41 -> OC119C5 38 -> OC139A1 11 -> OCC96E 5 -> OCA856 7 -> OC138EA 30 -> OC133F3 24 -> OC13028 33 -> OCAEC8 32 -> OC1355F 27 -> OC3D5C 31 -> OCB6A1 40 -> OCC1EE 46 -> OCA7AB 45 -> OC138E9 48 -> OC10A07 49 -> OC10A2F 47 -> OC10246 37 -> OC10A5C 35 -> OC3A6D 44 -> OC10A2D 39 -> OC110E6 43 -> OC131EE 36 -> OC14AD9 21 -> OC10DD3 23 -> OC10A28 20 -> OC13760 18 -> OC10164 19 -> OC10DA2 25 -> OC12CDC 22 -> OC13638 12 -> OC56B1 28 -> OC131FC 34 -> OC15301 26 -> OC157D3 17 -> OC13624 4 -> OC146FF 0
Distance of the route: 28990m



We can now get directions for 2 different routes, one where the geocaches are in random (original) order, and one with the optimal order. The routes are visualized on the map.

In [20]:
from openrouteservice import directions


def style_function(color):
    return lambda feature: dict(color=color, weight=3, opacity=1)

request = {'coordinates': locations,
           'profile': 'foot-walking',
           'geometry': 'true',
           'format_out': 'geojson',
           'instructions': 'false'          
          }
random_route = clnt.directions(**request)

folium.features.GeoJson(data=random_route,
                        name='Random route',
                        style_function=style_function('#84e184'),
                        overlay=True).add_to(m)

# And now the optimal route
request['coordinates'] = optimal_coords
optimal_route = clnt.directions(**request)
folium.features.GeoJson(data=optimal_route,
                        name='Optimal route',
                        style_function=style_function('#6666ff'),
                        overlay=True).add_to(m)

m.add_child(folium.map.LayerControl())
m

Lastly, we calculate the duration of each route.

In [21]:
optimal_duration = optimal_route['features'][0]['properties']['summary']['duration'] / 60
random_duration = random_route['features'][0]['properties']['summary']['duration'] / 60
    
print(f"Duration optimal route: {optimal_duration:.3f} mins\nDuration random route: {random_duration:.3f} mins")

Duration optimal route: 341.515 mins
Duration random route: 950.615 mins
